# Random velocity gradient

Random velocity gradient sampled from standard distribution, followed by relaxation. The velocity gradient is normalised. The Henky strain `H` is the duration of the velocity gradient event. 

In [1]:
import numpy as np
np.random.seed(2022)
from scipy.interpolate import interp1d
import pywt

from ipywidgets import interactive
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]

output_file = 'inputs/random/scenario_random'

# Number of independent scenarios
n_scenarios = 5

# Henky strain
H = 10

# Relax after strain
relax = False

## Sample velocity gradient data

In [2]:
if relax:
    times = [[0,H,2*H]]*n_scenarios
else:
    times = [[0,H]]*n_scenarios
gradUs = []

for i in range(n_scenarios):
    gradU = np.random.standard_normal(size=(3,3))
    gradU = gradU - (gradU[0,0] + gradU[1,1] + gradU[2,2])/3*np.eye(3)
    if relax:
        gradUs.append([gradU, np.zeros((3,3)), np.zeros((3,3))])
    else:
        gradUs.append([gradU, np.zeros((3,3))])

## Analyse strain

In [3]:
gradUs_norm = []

for gradU in gradUs:

    # Get principal values for the strain
    D = 0.5*(gradU + np.transpose(gradU, axes=(0,2,1)))
    strains = np.linalg.eigvalsh(D)
    max_Wi = np.max(strains[:,2])
    
    # Normalize
    gradUs_norm.append(gradU/max_Wi)
    print(f"Maximum strain rate: {max_Wi}")

Maximum strain rate: 1.1216058604085946
Maximum strain rate: 1.4088412068120273
Maximum strain rate: 1.2753193864897365
Maximum strain rate: 1.1061703910868559
Maximum strain rate: 1.2860439642528507


In [4]:
gradUs_norm

[array([[[-0.19515543, -0.24509628, -0.12418405],
         [ 1.76950409,  0.05683793,  0.67832087],
         [ 0.26834882,  0.48171759,  0.1383175 ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]),
 array([[[ 0.73719529, -0.06403361, -1.63676592],
         [ 0.81113472, -0.6209902 , -0.61309395],
         [ 0.72154685,  0.73391088, -0.11620509]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]),
 array([[[ 0.19140341, -0.30058632, -0.23851709],
         [ 0.78199353,  0.07678216, -1.15726768],
         [-1.52189824,  0.65367855, -0.26818558]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]),
 array([[[ 0.77457567,  0.28844456,  0.17255066],
         [ 0.33382758, -0.378623

In [5]:
for i, (time, gradU) in enumerate(zip(times, gradUs_norm)):
    np.savez_compressed(f"{output_file}_{i}", t=time, gradU=gradU)